# Imports

In [2]:
#importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
import duckdb

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
#importing data sets for X
forest_area = pd.read_csv('data/forest_area.csv',thousands=',')
health_expenditure = pd.read_csv('data/health_expenditure.csv',thousands=',')
tuberculosis = pd.read_csv('data/tuberculosis.csv',thousands=',')
urban_pop = pd.read_csv('data/urban_pop.csv',thousands=',')
dollar2_poverty = pd.read_csv('data/dollar2_poverty.csv',thousands=',')
dollar3_poverty = pd.read_csv('data/dollar3_poverty.csv',thousands=',')
current_expenditure = pd.read_csv('data/current_expenditure.csv', thousands = ',')
out_of_pocket = pd.read_csv('data/out_of_pocket.csv',thousands=',')
private_expenditure = pd.read_csv('data/private_expenditure.csv',thousands=',')
cooking_fuel_access = pd.read_csv('data/cooking_fuel_access.csv',thousands=',')
air_pollution = pd.read_csv('data/air_pollution.csv',thousands=',')
physician = pd.read_csv('data/physician.csv',thousands=',')
poverty_10 = pd.read_csv('data/poverty_10.csv',thousands=',')

In [3]:
#importing data set for y
y = pd.read_csv('./data/cancer.csv',thousands=',')

# Cleaning

The dropNclean function gets rid of columns that do not contain relevant data or are too sparse.

In [4]:
def dropNclean(df):
    df = df.drop(columns=map(str,range(1960,2000)))
    df = df.replace('..',np.NaN)
    return df.dropna(thresh=len(df.columns)*0.7)

## Cleaning X

### cleaning intermediary data sets

In [5]:
data_names = ['Proportion of Population Pushed Below $2.15 Poverty Line by Out-of-Pocket Health Care Expenditure (%)',
              'Proportion of Population Pushed Below $3.65 Poverty Line by Out-of-Pocket Health Care Expenditure (%)',
              'PM2.5 Air Pollution, Mean Annual Exposure (Mcg per Cubic Meter)',
              'Current Health Expenditure (% of GDP)',
              'Forest Area Coverage (%)',
              'Domestic General Government Health Expenditure per Capita (Current US$)',
              'Out-of-Pocket Ependiture per Capita (Current US$)',
              'Physicians (Per 1000 People)',
              'Proportion of Population Spending More Than 10% of Income on Healthcare (%)',
              'Private Health Expenditure per Capita (Current US$)',
              'Incidence of Tuberculosis (Per 100,000 People)',
              'Urban Population (% of Total Population)'
              ]

In [6]:
c_forest_area = dropNclean(forest_area)
c_health_expenditure = dropNclean(health_expenditure)
c_tuberculosis = dropNclean(tuberculosis)
c_urban_pop = dropNclean(urban_pop)
c_dollar2_poverty = dropNclean(dollar2_poverty)
c_dollar3_poverty = dropNclean(dollar3_poverty)
c_current_expenditure = dropNclean(current_expenditure)
c_out_of_pocket = dropNclean(out_of_pocket)
c_private_expenditure = dropNclean(private_expenditure)
c_cooking_fuel_access = dropNclean(cooking_fuel_access)
c_air_pollution = dropNclean(air_pollution)
c_physician = dropNclean(physician)
c_poverty_10 = dropNclean(poverty_10)

In [7]:
cleaned_data = [c_urban_pop, c_dollar2_poverty, c_dollar3_poverty, c_air_pollution,
                  c_current_expenditure, c_forest_area,
                  c_health_expenditure, c_out_of_pocket, c_physician,
                  c_poverty_10, c_private_expenditure, c_tuberculosis]

In [8]:
years = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009',
         '2010','2011','2012','2013','2014','2015','2016','2017','2017','2018','2019']

In [9]:
def rename_melt(df, stat):
    df.columns.values[0] = 'Country'

    return df.melt(id_vars=['Country'],
                   value_vars=years,
                   var_name='Year',
                   value_name=stat)

In [10]:
for i in range(len(cleaned_data)):
    cleaned_data[i] = rename_melt(cleaned_data[i], data_names[i])
    cleaned_data[i].iloc[:,-1] = cleaned_data[i].iloc[:,-1].apply(pd.to_numeric, errors='coerce').replace(",","")

In [11]:
cleaned_data_names = [ 'c_dollar2_poverty', 'c_dollar3_poverty', 'c_air_pollution',
                  'c_current_expenditure', 'c_forest_area',
                  'c_health_expenditure', 'c_out_of_pocket', 'c_physician',
                  'c_poverty_10', 'c_private_expenditure', 'c_tuberculosis', 'c_urban_pop']

In [12]:
# export cleaned data to csv files
for i in range(len(cleaned_data)):
    cleaned_data[i].to_csv('./data/cleaned_data/' + cleaned_data_names[i], index=False)

### creating one big data frame

In [13]:
X = cleaned_data[0]
for df in cleaned_data[1:]:
    X = pd.merge(X, df, on=['Country','Year'], how='left')

In [14]:
rename_col = {}
for i in np.array(range(12)):
    rename_col[data_names[i]] = cleaned_data_names[i]
rename_col['Country'] = 'country'
rename_col['Year'] = 'year'

renaming columns to make them easier to handle in python

In [15]:
X = X.rename(columns = rename_col)
print(X.columns)

Index(['country', 'year', 'c_dollar2_poverty', 'c_dollar3_poverty',
       'c_air_pollution', 'c_current_expenditure', 'c_forest_area',
       'c_health_expenditure', 'c_out_of_pocket', 'c_physician',
       'c_poverty_10', 'c_private_expenditure', 'c_tuberculosis',
       'c_urban_pop'],
      dtype='object')


getting rid of the countries without enough data (go from 5260 to 3669 rows), 7 was an arbitrary choice

In [16]:
X = X.dropna(axis=0, thresh=7)
print(X.shape)

(3669, 14)


getting rid of the columns that is too sparse on data, 70% filled was an arbitrary choice

In [17]:
X= X.dropna(axis=1, thresh =(3669*.7))
print(X.shape)

(3669, 9)


In [18]:
X.to_csv('./data/dataframes/X',index=False)

## Cleaning y

In [19]:
y = y.rename(columns = {' ' : 'country'})
y = dropNclean(y)
y = y.drop(columns = 'Unnamed: 11').set_index('country')
y = y[years].astype('float64')
y = y.reset_index()

In [20]:
y.to_csv('./data/dataframes/y')

# Supplementary DataFrames

## Supplementary X DataFrames

In [21]:
dict_of_year_dfs = {}
for yr in years:
    dict_of_year_dfs[yr] = X[X["year"] == str(yr)]

In [22]:
countries = X['country'].unique()

In [23]:
dict_of_country_dfs = {}
for c in countries:
    dict_of_country_dfs[c] = X[X['country'] == c]

In [24]:
for yr in years:
    X[X["year"] == str(y)].to_csv(f'./data/dataframes/data_by_year/X{yr}')

In [25]:
countries = X['country'].unique()

In [26]:
for c in countries:
    X[X['country'] == c].to_csv(f'./data/dataframes/data_by_country/X{c}')

In [27]:
X.columns

Index(['country', 'year', 'c_dollar2_poverty', 'c_forest_area',
       'c_health_expenditure', 'c_out_of_pocket', 'c_physician',
       'c_tuberculosis', 'c_urban_pop'],
      dtype='object')

## Supplementary Stat DataFrames

In [28]:
country_means_dict = {}
country_medians_dict = {}
country_variances_dict = {}
columns = dict_of_country_dfs['Bulgaria'].columns[2:]

In [29]:
hold = y.set_index('country')
hold = hold.transpose()
y_stat = pd.DataFrame(data = [hold.mean(), hold.median(), hold.var()]).transpose().reset_index()
y_stat = y_stat.rename(columns = {0 : 'y_mean', 1 : 'y_median' , 2 : 'y_variance'})
y_stat.to_csv('./data/dataframes/y_stat')

In [30]:
for c in countries:
    country_means_dict[c] = dict_of_country_dfs[c].get(columns).mean()
    country_medians_dict[c] = dict_of_country_dfs[c].get(columns).mean()
    country_variances_dict[c] = dict_of_country_dfs[c].get(columns).mean()

In [31]:
country_means_df = pd.DataFrame(data=country_means_dict).transpose()
country_means_df = country_means_df.reset_index().rename(columns = {'index' : 'country'})
country_means_df = pd.merge(country_means_df,y_stat,left_on = ['country'], right_on = ['country'])
country_means_df.to_csv('./data/dataframes/country_means')

In [32]:
country_medians_df = pd.DataFrame(data=country_medians_dict).transpose()
country_medians_df = country_medians_df.reset_index().rename(columns = {'index' : 'country'})
country_medians_df = pd.merge(country_medians_df,y_stat,left_on = ['country'], right_on = ['country'])
country_medians_df.to_csv('./data/dataframes/country_medians')

In [33]:
country_variances_df = pd.DataFrame(data=country_variances_dict).transpose()
country_variances_df = country_variances_df.reset_index().rename(columns = {'index' : 'country'})
country_variances_df = pd.merge(country_variances_df,y_stat,left_on = ['country'], right_on = ['country'])
country_variances_df.to_csv('./data/dataframes/country_variances')

In [6]:
aus_1 = pd.read_csv('./data/aus_1yr.csv')
aus_3 = pd.read_csv('./data/aus_3yr.csv')
aus_5 = pd.read_csv('./data/aus_5yr.csv')

can_1 = pd.read_csv('./data/cana_1yr.csv')
can_3 = pd.read_csv('./data/cana_3yr.csv')
can_5 = pd.read_csv('./data/cana_5yr.csv')

den_1 = pd.read_csv('./data/den_1yr.csv')
den_3 = pd.read_csv('./data/den_3yr.csv')
den_5 = pd.read_csv('./data/den_5yr.csv')

ire_1 = pd.read_csv('./data/ire_1yr.csv')
ire_3 = pd.read_csv('./data/ire_3yr.csv')
ire_5 = pd.read_csv('./data/ire_5yr.csv')

newz_1 = pd.read_csv('./data/newz_1yr.csv')
newz_3 = pd.read_csv('./data/newz_3yr.csv')
newz_5 = pd.read_csv('./data/newz_5yr.csv')

nor_1 = pd.read_csv('./data/nor_1yr.csv')
nor_3 = pd.read_csv('./data/nor_3yr.csv')
nor_5 = pd.read_csv('./data/nor_5yr.csv')

uk_1 = pd.read_csv('./data/uk_1yr.csv')
uk_3 = pd.read_csv('./data/uk_3yr.csv')
uk_5 = pd.read_csv('./data/uk_5yr.csv')


In [11]:
year_1 = np.concatenate([aus_1,can_1,den_1,ire_1,newz_1,nor_1,uk_1],axis=1)
print(year_1)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 20 and the array at index 3 has size 19

In [38]:
hdi = pd.read_csv('./data/hdi_time_series.csv')
hdi.head()


,iso3,country,hdicode,region,hdi_rank_2021,hdi_1990,hdi_1991,hdi_1992,hdi_1993,hdi_1994,...,mf_2012,mf_2013,mf_2014,mf_2015,mf_2016,mf_2017,mf_2018,mf_2019,mf_2020,mf_2021
0,AFG,Afghanistan,Low,SA,180.0,0.273,0.279,0.287,0.297,0.292,...,1.86,1.88,1.66,1.62,1.66,1.41,1.32,1.38,1.38,1.38
1,AGO,Angola,Medium,SSA,148.0,NaN,NaN,NaN,NaN,NaN,...,4.09,4.53,3.97,3.59,2.79,2.64,2.28,2.18,2.18,2.18
2,ALB,Albania,High,ECA,67.0,0.647,0.629,0.614,0.617,0.624,...,12.44,11.49,13.14,12.61,14.39,14.46,12.85,12.96,12.96,12.96
3,AND,Andorra,Very High,NaN,40.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ARE,United Arab Emirates,Very High,AS,26.0,0.728,0.739,0.742,0.748,0.755,...,49.56,49.68,55.49,59.76,64.95,75.61,65.97,68.95,68.95,68.95


In [39]:
hdi = hdi[['country','hdicode']]
mapping = {'Low':1,'Medium':2,'High':3,'Very High':4}
hdi['hdicode'] = hdi['hdicode'].replace(mapping)
hdi.head(20)
hdi.to_csv('./data/dataframes/hdi')